第 1 步：定义 Schema 字段

In [10]:
# from pymilvus import Collection, CollectionSchema, FieldSchema, DataType
from pymilvus import connections, MilvusClient, DataType, Function, FunctionType

In [6]:
# Initialize Milvus client
client = MilvusClient(
    uri="http://localhost:19530",
)

# Create a new schema for the collection
schema = client.create_schema()

第 2 步：向 Schema 添加嵌入函数

In [7]:
# Add primary field "id"
schema.add_field("id", DataType.INT64, is_primary=True, auto_id=False)

# Add scalar field "document" for storing textual data
schema.add_field("document", DataType.VARCHAR, max_length=9000)

# Add vector field "dense" for storing embeddings.
# IMPORTANT: Set dim to match the exact output dimension of the embedding model.
# For instance, OpenAI's text-embedding-3-small model outputs 1536-dimensional vectors.
# For dense vector, data type can be FLOAT_VECTOR or INT8_VECTOR
schema.add_field("dense", DataType.FLOAT_VECTOR, dim=1536)

{'auto_id': False, 'description': '', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'document', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 9000}}, {'name': 'dense', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 1536}}], 'enable_dynamic_field': False}

In [8]:
# Define embedding function (example: OpenAI provider)
text_embedding_function = Function(
    name="openai_embedding",                  # Unique identifier for this embedding function
    function_type=FunctionType.TEXTEMBEDDING, # Type of embedding function
    input_field_names=["document"],           # Scalar field to embed
    output_field_names=["dense"],             # Vector field to store embeddings
    params={                                  # Provider-specific configuration (highest priority)
        "provider": "openai",                 # Embedding model provider
        "model_name": "text-embedding-3-small",     # Embedding model
        # "credential": "apikey1",            # Optional: Credential label
        # Optional parameters:
        # "dim": "1536",       # Optionally shorten the vector dimension
        # "user": "user123"    # Optional: identifier for API tracking
    }
)

# Add the embedding function to your schema
schema.add_function(text_embedding_function)

{'auto_id': False, 'description': '', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'document', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 9000}}, {'name': 'dense', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 1536}, 'is_function_output': True}], 'enable_dynamic_field': False, 'functions': [{'name': 'openai_embedding', 'description': '', 'type': <FunctionType.TEXTEMBEDDING: 2>, 'input_field_names': ['document'], 'output_field_names': ['dense'], 'params': {'provider': 'openai', 'model_name': 'text-embedding-3-small'}}]}

第 3 步：配置索引

In [9]:
# Prepare index parameters
index_params = client.prepare_index_params()

# Add AUTOINDEX to automatically select optimal indexing method
index_params.add_index(
    field_name="dense",
    index_type="AUTOINDEX",
    metric_type="COSINE" 
)

第 4 步：创建 Collections

In [11]:
connections.connect(
    alias="default",
    host='localhost',
    port='19530',
    db_name='my_database_1'  # 指定要操作的数据库
)

# Create collection named "demo"
client.create_collection(
    collection_name='demo', 
    schema=schema, 
    index_params=index_params
)

2025-12-30 11:06:57,769 [ERROR][handler]: RPC error: [create_collection], <MilvusException: (code=65535, message=Missing credentials config or configure the MILVUS_OPENAI_API_KEY environment variable in the Milvus service.)>, <Time:{'RPC start': '2025-12-30 11:06:57.760051', 'RPC error': '2025-12-30 11:06:57.767388'}>
Traceback:
Traceback (most recent call last):
  File "a:\programmers\anaconda3\Lib\site-packages\pymilvus\decorators.py", line 263, in handler
    return func(*args, **kwargs)
  File "a:\programmers\anaconda3\Lib\site-packages\pymilvus\decorators.py", line 322, in handler
    return func(self, *args, **kwargs)
  File "a:\programmers\anaconda3\Lib\site-packages\pymilvus\decorators.py", line 196, in handler
    raise e from e
  File "a:\programmers\anaconda3\Lib\site-packages\pymilvus\decorators.py", line 166, in handler
    return func(*args, **kwargs)
  File "a:\programmers\anaconda3\Lib\site-packages\pymilvus\client\grpc_handler.py", line 381, in create_collection
    ch

MilvusException: <MilvusException: (code=65535, message=Missing credentials config or configure the MILVUS_OPENAI_API_KEY environment variable in the Milvus service.)>